In [2]:
# Import Modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('dark_background')

%load_ext autoreload
%autoreload 2

pd.set_option('display.min_row', 10)
pd.set_option('display.max_column', 100)
pd.set_option('display.max_colwidth', 300)

In [3]:
streets = pd.read_csv('../data/Seattle_Streets.csv')
collisions = pd.read_csv('../data/Collisions.csv')
crosswalks = pd.read_csv('../data/Marked_Crosswalks.csv')

In [5]:
crosswalks.head()

,X,Y,OBJECTID,COMPKEY,UNITID,UNITDESC,CONDITION,CONDITION_ASSESSMENT_DATE,OWNERSHIP,CURRENT_STATUS,PRIMARYDISTRICTCD,SECONDARYDISTRICTCD,OVERRIDEYN,COMPTYPE,SEGKEY,UNITTYPE,OLD_ID,APPROACH,MARKING_TYPE,SCHOOL,MIDBLOCK_CROSSWALK,INSTALL_DATE,COLOR,COMMENTS,CATEGORY,OWNERSHIP_DATE,CURRENT_STATUS_DATE,MAINTAINED_BY,MAINTENANCE_AGREEMENT,CURBSPACEID,MAINT_DISTRICT,OVERRIDECOMMENT,SHAPE_LNG,SHAPE_LAT,CROSSWALK_CONTROL,MATERIAL,ONSTREET,XSTRLOW,MEAS_FROM_LOW,DISTTOLOW,STPOINT,XSTRHI,MEAS_FROM_HI,DISTTOHI,OFFSET,SIDE,MEASUREMENT_ORIGIN,ATTACHMENT_1,ATTACHMENT_2,ATTACHMENT_3,ATTACHMENT_4,ATTACHMENT_5,ATTACHMENT_6,ATTACHMENT_7,ATTACHMENT_8,ATTACHMENT_9,MAINTENANCE_GROUP,NUM_ATTACHMENTS
0,1.269560e+06,223140.574842,1,522399,XWK-3325,ALASKAN WAY 0060 BLOCK C SIDE ( 35) 35 FT NW/O YESLER WAY,GOOD,2012/09/30 00:00:00+00,,INSVC,DISTRICT7,,N,16,8717,XWK,Old ID # 758,NW,LADER,N,N,1970/01/01 00:00:00+00,WHT,"ALASKAN WAY & YESLER WAY, N",XWK,1970/01/01 00:00:00+00,2009/01/22 00:00:00+00,,NaN,NaN,CENTRAL,NaN,-122.336656,47.601800,,THRPL,ALASKAN WAY,YESLER WAY,Y,35,0,COLUMBIA ST,N,234,0,C,XStreet Centerline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XWK-CD7-YEAR4,0
1,1.283979e+06,201215.071449,2,525064,XWK-3029,S HOLLY ST 0440 BLOCK C SIDE ( 385) 20 FT W/O 45TH AVE S,GOOD,2013/08/26 00:00:00+00,,INSVC,DISTRICT2,,N,16,20363,XWK,Old ID # 662,W,LADER,Y,N,1970/01/01 00:00:00+00,WHT,"WI #13959 8/27/98 <br> School # 8<br>45 AVE S & S HOLLY ST, W",XWK,1970/01/01 00:00:00+00,2008/11/14 00:00:00+00,,NaN,NaN,SOUTH,NaN,-122.276553,47.542459,,THRPL,S HOLLY ST,44TH AVE S,N,385,0,45TH AVE S,Y,20,0,C,XStreet Centerline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XWK-CD2-YEAR4,0
2,1.268498e+06,242364.672876,3,523429,XWK-416,STONE WAY N 0390 BLOCK C SIDE ( 115) 115 FT N/O N 39TH ST,GOOD,2017/06/29 00:00:00+00,SDOT,INSVC,DISTRICT4,,N,16,12959,XWK,Old ID # 485,S,LADER,N,N,1970/01/01 00:00:00+00,WHT,"<p>WI #18482 <br>STONE WAY N &amp; BRIDGE WAY N, S</p><p><br></p><p>Remarked per WO 528218</p>",XWK,1970/01/01 00:00:00+00,2009/02/10 00:00:00+00,SDOT,NaN,NaN,NORTH,NaN,-122.342486,47.654435,SIGNAL,THRPL,STONE WAY N,N 39TH ST,Y,115,0,BRIDGE WAY N,N,83,0,C,XStreet Centerline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XWK-CD4-YEAR1,0
3,1.259573e+06,211119.144160,4,521447,XWK-2387,35TH AVE SW 0390 BLOCK C SIDE ( 18) 18 FT S/O SW ANDOVER N ST,GOOD,2019/07/23 00:00:00+00,SDOT,INSVC,DISTRICT1,,N,16,5446,XWK,Old ID # 162,S,LADER,Y,N,1970/01/01 00:00:00+00,WHT,"School # 74<br>35 AVE SW & SW ANDOVER ST (W LEG), S",XWK,1970/01/01 00:00:00+00,2009/01/06 00:00:00+00,,NaN,NaN,SOUTH,NaN,-122.376151,47.568308,,THRPL,35TH AVE SW,SW ANDOVER N ST,Y,18,0,SW ANDOVER S ST,N,112,0,C,XStreet Centerline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XWK-CD1-YEAR3,0
4,1.277718e+06,271174.016176,5,524665,XWK-1892,NE 145TH ST 0200 BLOCK C SIDE ( 5) 5 FT E/O 20TH AVE NE,GOOD,2012/06/15 00:00:00+00,,INSVC,DISTRICT5,,N,16,16688,XWK,Old ID # 505,N,LADER,N,N,1970/01/01 00:00:00+00,WHT,"WI #18482 <br>City Limits<br>20 AVE NE & NE 145 ST, N",XWK,1970/01/01 00:00:00+00,2012/12/13 00:00:00+00,,NaN,NaN,NORTH,NaN,-122.307324,47.733890,,THRPL,NE 145TH ST,20TH AVE NE,Y,5,0,22ND AVE NE,N,328,0,C,XStreet Centerline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XWK-CD5-YEAR4,0


2065 unique blocks are in the collisions data (meaning 2065 has had an incident over the years)

In [12]:
collisions[collisions['CROSSWALKKEY'].isin(crosswalks['COMPKEY'])].groupby('CROSSWALKKEY').count()

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,HITPARKEDCAR
CROSSWALKKEY,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
520756,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
520757,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1
520758,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1
520762,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,1,1,1
520764,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701110,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,1,1,1
701280,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,1,1,1
701306,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1


In [14]:
crosswalks[crosswalks['COMPKEY'].isin(collisions['CROSSWALKKEY'])].groupby('COMPKEY').count()

,X,Y,OBJECTID,UNITID,UNITDESC,CONDITION,CONDITION_ASSESSMENT_DATE,OWNERSHIP,CURRENT_STATUS,PRIMARYDISTRICTCD,SECONDARYDISTRICTCD,OVERRIDEYN,COMPTYPE,SEGKEY,UNITTYPE,OLD_ID,APPROACH,MARKING_TYPE,SCHOOL,MIDBLOCK_CROSSWALK,INSTALL_DATE,COLOR,COMMENTS,CATEGORY,OWNERSHIP_DATE,CURRENT_STATUS_DATE,MAINTAINED_BY,MAINTENANCE_AGREEMENT,CURBSPACEID,MAINT_DISTRICT,OVERRIDECOMMENT,SHAPE_LNG,SHAPE_LAT,CROSSWALK_CONTROL,MATERIAL,ONSTREET,XSTRLOW,MEAS_FROM_LOW,DISTTOLOW,STPOINT,XSTRHI,MEAS_FROM_HI,DISTTOHI,OFFSET,SIDE,MEASUREMENT_ORIGIN,ATTACHMENT_1,ATTACHMENT_2,ATTACHMENT_3,ATTACHMENT_4,ATTACHMENT_5,ATTACHMENT_6,ATTACHMENT_7,ATTACHMENT_8,ATTACHMENT_9,MAINTENANCE_GROUP,NUM_ATTACHMENTS
COMPKEY,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
520756,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1
520757,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1
520758,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1
520762,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1
520764,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701110,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1
701280,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1
701306,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1


## Clean up crosswalks dataset

In [16]:
crosswalks.columns = crosswalks.columns.map(str.lower)

In [17]:
crosswalks.columns

Index(['x', 'y', 'objectid', 'compkey', 'unitid', 'unitdesc', 'condition',
       'condition_assessment_date', 'ownership', 'current_status',
       'primarydistrictcd', 'secondarydistrictcd', 'overrideyn', 'comptype',
       'segkey', 'unittype', 'old_id', 'approach', 'marking_type', 'school',
       'midblock_crosswalk', 'install_date', 'color', 'comments', 'category',
       'ownership_date', 'current_status_date', 'maintained_by',
       'maintenance_agreement', 'curbspaceid', 'maint_district',
       'overridecomment', 'shape_lng', 'shape_lat', 'crosswalk_control',
       'material', 'onstreet', 'xstrlow', 'meas_from_low', 'disttolow',
       'stpoint', 'xstrhi', 'meas_from_hi', 'disttohi', 'offset', 'side',
       'measurement_origin', 'attachment_1', 'attachment_2', 'attachment_3',
       'attachment_4', 'attachment_5', 'attachment_6', 'attachment_7',
       'attachment_8', 'attachment_9', 'maintenance_group', 'num_attachments'],
      dtype='object')

In [ ]:
# add a column for gps coordinates in order to match crosswalks and collisions based on gps

blocks['gps'] = [(x,y) for x,y in zip(round(blocks['Y'], 5), round(blocks['X'], 5))]
crosswalks['gps'] = [(x,y) for x,y in zip(round(crosswalks['SHAPE_LAT'], 5), round(crosswalks['SHAPE_LNG'], 5))]